In [4]:
#导入相关的库
import mysql.connector
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta
from dateutil.parser import parse

In [5]:
#数据库连接初始化
mydb = mysql.connector.connect(
    host="rm-bp178mt2593im687xjo.mysql.rds.aliyuncs.com",
    user="root",
    passwd="KLxs8888",
    database="wdgj"
)
print(mydb)

In [6]:
# mydb = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     passwd="qaz!@wsx",
#     database="work"
# )
# print(mydb)

In [7]:
#定义SQL语句
profitCheck='SELECT * FROM profit_check_sku1901'
returnDetail ='SELECT * FROM return_sku1901'

In [8]:
#打开游标，获取数据
mycursor=mydb.cursor()
mycursor.execute(profitCheck)
profitCheck1901=mycursor.fetchall()
mycursor.close()

True

In [9]:
#打开游标，获取数据
mycursor=mydb.cursor()
mycursor.execute(returnDetail)
returnSku1901=mycursor.fetchall()
mycursor.close()

True

In [10]:
#数据格式转化成为pandas
profitCheck1901=pd.DataFrame(profitCheck1901)
returnSku1901=pd.DataFrame(returnSku1901)

In [11]:
#修改列名
profitCheck1901.columns=['Date','GoodsNO','BarCode','GoodsName','GoodsCount','Income','Cost','FavourableTotal'\
                        ,'Postage','PostageTotal','OperationCost']
returnSku1901.columns=['Date','GoodsNO','GoodsName','ReturnTotal','GoodsCount','returnCost']

In [12]:
returnSku1901["GoodsCount"]=returnSku1901["GoodsCount"].astype(float)
result2=returnSku1901.groupby(['Date'],as_index=False)
result2=result2.aggregate(np.sum)#出库总数
result2.GoodsCount.sum()

30145.620000000003

In [13]:
profitCheck1901.head()

,Date,GoodsNO,BarCode,GoodsName,GoodsCount,Income,Cost,FavourableTotal,Postage,PostageTotal,OperationCost
0,2019-01,C.WB.02.01.000052,6925125561387,KUB可优比-儿童保温杯-绿色,6255.0,675578.00,226243.35,277114.77,24065.26,195.23,18184.08
1,2019-01,C.TC.01.01.000059,6925125592633,KUB可优比-艾迪斯婴儿床-新款白马FD826-120*65,1552.0,1369690.07,798038.40,178946.00,81445.20,3667.00,4641.68
2,2019-01,D04853501,6971235895140,蒂爱-水杯餐具礼盒装-常规款,1661.0,425460.20,191779.06,173415.91,8950.39,0.00,4935.84
3,2019-01,C.WJ.01.01.000021,6925125559629,KUB可优比-三合一多功能电子琴,2145.0,267518.80,111743.78,127649.38,32843.74,603.00,6299.47
4,2019-01,K04857901,6971971887126,KUB可优比-恐龙滑梯组合,3818.0,1159592.00,622334.00,111138.71,119292.60,4111.00,11445.00


In [14]:
#合并出库明细和退货明细
data_merge = pd.merge(profitCheck1901,returnSku1901,on=['Date','GoodsNO'],how='left')

In [15]:
data_merge.head()

,Date,GoodsNO,BarCode,GoodsName_x,GoodsCount_x,Income,Cost,FavourableTotal,Postage,PostageTotal,OperationCost,GoodsName_y,ReturnTotal,GoodsCount_y,returnCost
0,2019-01,C.WB.02.01.000052,6925125561387,KUB可优比-儿童保温杯-绿色,6255.0,675578.00,226243.35,277114.77,24065.26,195.23,18184.08,KUB可优比-儿童保温杯-绿色,18251.40,235.0,8499.95
1,2019-01,C.TC.01.01.000059,6925125592633,KUB可优比-艾迪斯婴儿床-新款白马FD826-120*65,1552.0,1369690.07,798038.40,178946.00,81445.20,3667.00,4641.68,KUB可优比-艾迪斯婴儿床-新款白马FD826-120*65,23919.54,29.0,14911.80
2,2019-01,D04853501,6971235895140,蒂爱-水杯餐具礼盒装-常规款,1661.0,425460.20,191779.06,173415.91,8950.39,0.00,4935.84,蒂爱-水杯餐具礼盒装-常规款,8562.80,40.0,4618.40
3,2019-01,C.WJ.01.01.000021,6925125559629,KUB可优比-三合一多功能电子琴,2145.0,267518.80,111743.78,127649.38,32843.74,603.00,6299.47,KUB可优比-三合一多功能电子琴,9737.49,110.0,5730.45
4,2019-01,K04857901,6971971887126,KUB可优比-恐龙滑梯组合,3818.0,1159592.00,622334.00,111138.71,119292.60,4111.00,11445.00,KUB可优比-恐龙滑梯组合,20819.51,73.0,11899.00


In [16]:
returnSku1901["GoodsCount"]=returnSku1901["GoodsCount"].astype(float)
data_merge1=data_merge.groupby(['Date'],as_index=False)
data_merge1=data_merge1.aggregate(np.sum)#出库总数
data_merge1.GoodsCount_y.sum()

29513.620000000006

In [17]:
data_merge.to_excel("data_merge_right_sku.xlsx",header=True)

In [18]:
del data_merge['GoodsName_y']

In [19]:
#获取商品小组对应情况
GoodsInfo =pd.read_csv('C:\\Users\\Administrator\\PycharmProjects\\test\\goodsinfo0305.csv',header = 0,engine='python')
GoodsInfo.columns=['GoodsNO','GoodsName','barCode','Company','Dept','TmallTax','GroupName','Holder','GradeTwo','GradeThird','GradeForth','SPU']

In [20]:
#删除重复以及不必要列
GoodsInfoShort=GoodsInfo[['GoodsNO','Company','Dept','GroupName','TmallTax','Holder','GradeTwo','GradeThird','GradeForth','SPU']]

In [21]:
GoodsInfoShort.shape

(17337, 10)

In [22]:
#以SKU编码为唯一辨识，去重
GoodsInfoShort=GoodsInfoShort.drop_duplicates(["GoodsNO"])

In [23]:
GoodsInfoShort.shape

(17203, 10)

In [24]:
GoodsInfoShort.head()

,GoodsNO,Company,Dept,GroupName,TmallTax,Holder,GradeTwo,GradeThird,GradeForth,SPU
0,K04855901,可优比母婴用品有限公司,KUB穿戴,礼盒配饰组,0.05,谢涛涛,婴童配饰,婴童饰品,领结,K050231
1,K04855902,可优比母婴用品有限公司,KUB穿戴,礼盒配饰组,0.05,谢涛涛,婴童配饰,婴童饰品,领结,K050231
2,K04852202,可优比母婴用品有限公司,KUB穿戴,内服组,0.05,王江峰,婴童外衣,外套/夹克/大衣,其它外套,K050209
3,K04852201,可优比母婴用品有限公司,KUB穿戴,内服组,0.05,王江峰,婴童外衣,外套/夹克/大衣,其它外套,K050209
4,K04852205,可优比母婴用品有限公司,KUB穿戴,内服组,0.05,王江峰,婴童外衣,外套/夹克/大衣,其它外套,K050209


In [25]:
#匹配销售数据和对应小组信息
data_merge_dept=pd.merge(data_merge,GoodsInfoShort,on=['GoodsNO'],how='left')

In [26]:
# data_merge_dept = pd.read_csv('C:\\Users\\Administrator\\Desktop\\sell_return_dept.csv')

In [27]:
data_merge_dept.head()

,Date,GoodsNO,BarCode,GoodsName_x,GoodsCount_x,Income,Cost,FavourableTotal,Postage,PostageTotal,...,returnCost,Company,Dept,GroupName,TmallTax,Holder,GradeTwo,GradeThird,GradeForth,SPU
0,2019-01,C.WB.02.01.000052,6925125561387,KUB可优比-儿童保温杯-绿色,6255.0,675578.00,226243.35,277114.77,24065.26,195.23,...,8499.95,可优比母婴用品有限公司,KUB喂辅,学饮组,0.02,章小慧,学饮类,保温杯/壶,保温吸管杯,WF101
1,2019-01,C.TC.01.01.000059,6925125592633,KUB可优比-艾迪斯婴儿床-新款白马FD826-120*65,1552.0,1369690.07,798038.40,178946.00,81445.20,3667.00,...,14911.80,可优比母婴用品有限公司,KUB睡眠,睡床小组,0.05,林嘉欣,婴童卧室家具,婴童床,实木床,SL8512
2,2019-01,D04853501,6971235895140,蒂爱-水杯餐具礼盒装-常规款,1661.0,425460.20,191779.06,173415.91,8950.39,0.00,...,4618.40,杭州蒂爱母婴用品有限公司,蒂爱喂养,NaN,0.02,NaN,NaN,NaN,NaN,NaN
3,2019-01,C.WJ.01.01.000021,6925125559629,KUB可优比-三合一多功能电子琴,2145.0,267518.80,111743.78,127649.38,32843.74,603.00,...,5730.45,可优比母婴用品有限公司,KUB玩教,安抚玩乐组,0.05,何勉勉/郭凤,情景互动,音乐/乐器类,电子琴,WJ0308
4,2019-01,K04857901,6971971887126,KUB可优比-恐龙滑梯组合,3818.0,1159592.00,622334.00,111138.71,119292.60,4111.00,...,11899.00,可优比母婴用品有限公司,KUB玩教,学爬健身组,0.05,魏愿/陆昌胜,学步健身,游乐设施,滑梯,WJ0418


In [28]:
# kub=pd.read_csv('C:\\Users\\Administrator\\Desktop\\商品SKU事业部\\0124\\kub0125.csv',engine='python')
# kub=kub[['GoodsNO','Dept','GroupName']]
# kub=kub.drop_duplicates(["GoodsNO"])
# kub.columns=['SKU','Dept','GroupName']
# data_merge_dept_gp=pd.merge(data_merge_dept,kub,on=['SKU'],how='left')
# data_merge_dept_gp.to_excel('data_group_new0125.xlsx',header=True,index=False)

In [29]:
#修改合并后的列名
data_merge_dept.columns=['Date', 'SKU', 'BarCode', 'Name', 'SellCount', 'SellIncome',
       'SellCost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost',
       'ReturnTotal', 'ReturnCount', 'ReturnCost', 'Company', 'Dept',
       'GroupName','TmallTax','Holder','GradeTwo','GradeThird','GradeForth','SPU']

In [30]:
data_merge_dept.head()

,Date,SKU,BarCode,Name,SellCount,SellIncome,SellCost,FavourableTotal,Postage,PostageTotal,...,ReturnCost,Company,Dept,GroupName,TmallTax,Holder,GradeTwo,GradeThird,GradeForth,SPU
0,2019-01,C.WB.02.01.000052,6925125561387,KUB可优比-儿童保温杯-绿色,6255.0,675578.00,226243.35,277114.77,24065.26,195.23,...,8499.95,可优比母婴用品有限公司,KUB喂辅,学饮组,0.02,章小慧,学饮类,保温杯/壶,保温吸管杯,WF101
1,2019-01,C.TC.01.01.000059,6925125592633,KUB可优比-艾迪斯婴儿床-新款白马FD826-120*65,1552.0,1369690.07,798038.40,178946.00,81445.20,3667.00,...,14911.80,可优比母婴用品有限公司,KUB睡眠,睡床小组,0.05,林嘉欣,婴童卧室家具,婴童床,实木床,SL8512
2,2019-01,D04853501,6971235895140,蒂爱-水杯餐具礼盒装-常规款,1661.0,425460.20,191779.06,173415.91,8950.39,0.00,...,4618.40,杭州蒂爱母婴用品有限公司,蒂爱喂养,NaN,0.02,NaN,NaN,NaN,NaN,NaN
3,2019-01,C.WJ.01.01.000021,6925125559629,KUB可优比-三合一多功能电子琴,2145.0,267518.80,111743.78,127649.38,32843.74,603.00,...,5730.45,可优比母婴用品有限公司,KUB玩教,安抚玩乐组,0.05,何勉勉/郭凤,情景互动,音乐/乐器类,电子琴,WJ0308
4,2019-01,K04857901,6971971887126,KUB可优比-恐龙滑梯组合,3818.0,1159592.00,622334.00,111138.71,119292.60,4111.00,...,11899.00,可优比母婴用品有限公司,KUB玩教,学爬健身组,0.05,魏愿/陆昌胜,学步健身,游乐设施,滑梯,WJ0418


In [ ]:
#data_merge_dept['Postage']=0

In [ ]:
data_merge_dept['GroupName']=data_merge_dept['GroupName'].fillna('#N/A')

In [ ]:
department_gross_profit = pd.DataFrame()
department_gross_profit = pd.concat([department_gross_profit,data_merge_dept])

In [ ]:
#查看缺失值情况
department_gross_profit=department_gross_profit.fillna(np.nan)
department_gross_profit.isnull().sum()

In [ ]:
department_gross_profit.head()

In [ ]:
department_gross_profit[['SellCount', 'SellIncome',
       'SellCost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost',
       'ReturnTotal', 'ReturnCount', 'ReturnCost','TmallTax']] = department_gross_profit[['SellCount', 'SellIncome',
       'SellCost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost',
       'ReturnTotal', 'ReturnCount', 'ReturnCost','TmallTax']].astype(float)

In [ ]:
department_gross_profit.head()

In [ ]:
#填充缺失值为0
department_gross_profit[['ReturnTotal', 'ReturnCount', 'ReturnCost','TmallTax']]=department_gross_profit\
[['ReturnTotal', 'ReturnCount', 'ReturnCost','TmallTax']].fillna(0)

#填充缺失小组为#N/A
department_gross_profit['GroupName']=department_gross_profit\
['GroupName'].fillna('#N/A')

In [ ]:
department_gross_profit['CheckIncome'] = department_gross_profit['SellIncome']-department_gross_profit['FavourableTotal']\
                                        +department_gross_profit['PostageTotal']-department_gross_profit['ReturnTotal']
department_gross_profit['CheckCost'] = department_gross_profit['SellCost']-department_gross_profit['ReturnCost']
department_gross_profit['TmallCharge'] = (department_gross_profit['SellIncome']-department_gross_profit['FavourableTotal']-\
                                        department_gross_profit['ReturnTotal'])*department_gross_profit['TmallTax']

In [ ]:
#按公司，部门，日期进行归类
department_gross_profit_gb=department_gross_profit.groupby(['Company','Dept','Date'],as_index=False)

In [ ]:
#归类汇总
department_gross_profit_gb = department_gross_profit_gb.aggregate(np.sum)
del department_gross_profit_gb['TmallTax']

In [ ]:
#计算营业税，毛利润，毛利率
department_gross_profit_gb['Tax']=department_gross_profit_gb['CheckIncome']*0.06
department_gross_profit_gb['GrossProfit']=department_gross_profit_gb['CheckIncome']-department_gross_profit_gb['CheckCost']-\
                                            department_gross_profit_gb['CheckIncome']*0.06-department_gross_profit_gb['TmallCharge']-\
                                            department_gross_profit_gb['Postage']-department_gross_profit_gb['OperationCost']
department_gross_profit_gb['GrossProfitRate']=department_gross_profit_gb['GrossProfit']/department_gross_profit_gb['SellIncome']

In [ ]:
department_gross_profit_gb=department_gross_profit_gb[(department_gross_profit_gb['Dept']!='KUB包材')&(department_gross_profit_gb['Dept']!='KUB赠品')\
                          &(department_gross_profit_gb['Dept']!='蒂爱共用')]

In [ ]:
department_gross_profit_gb

In [ ]:
department_gross_profit_gb.to_csv("department_gross_profit_gb.csv",header=True,index=False)
department_gross_profit_gb.to_excel("department_gross_profit_gb1.xlsx",header=True,index=False)

In [ ]:
#按公司、部门、小组已经日期进行归类
department_gross_profit_gb_group=department_gross_profit.groupby(['Company','Dept','GroupName','Date'],as_index=False)

In [ ]:
department_gross_profit_gb_group = department_gross_profit_gb_group.aggregate(np.sum)
del department_gross_profit_gb_group['TmallTax']

In [ ]:
department_gross_profit_gb_group.head()

In [ ]:
department_gross_profit_gb_group['Tax']=department_gross_profit_gb_group['CheckIncome']*0.06
department_gross_profit_gb_group['GrossProfit']=department_gross_profit_gb_group['CheckIncome']-department_gross_profit_gb_group['CheckCost']-\
                                            department_gross_profit_gb_group['CheckIncome']*0.06-department_gross_profit_gb_group['TmallCharge']-\
                                            department_gross_profit_gb_group['Postage']-department_gross_profit_gb_group['OperationCost']
department_gross_profit_gb_group['GrossProfitRate']=department_gross_profit_gb_group['GrossProfit']/department_gross_profit_gb_group['SellIncome']

In [ ]:
department_gross_profit_gb_group=department_gross_profit_gb_group[(department_gross_profit_gb_group['Dept']!='KUB包材')&(department_gross_profit_gb_group['Dept']!='KUB赠品')\
                          &(department_gross_profit_gb_group['Dept']!='蒂爱共用')]

In [ ]:
department_gross_profit_gb_group.head()

In [ ]:
department_gross_profit_gb_group.to_csv("department_gross_profit_gb_group.csv",header=True,index=False)

In [31]:
data_merge_dept.to_csv("data_merge_dept.csv",header=True,index=False)

In [ ]:
data_merge_dept_edit=pd.DataFrame()
data_merge_dept_edit=pd.concat([data_merge_dept_edit,data_merge_dept])

In [ ]:
data_merge_dept_edit.columns=['日期','SKU编码','条码','品名','销售数量','销售收入','货品成本','优惠金额','快递费',\
                        '应收邮资','操作费','退货金额','退货数量','退货货品成本','公司名称','事业部','小组','天猫扣点','负责人','二级类目','三级类目','四级类目','SPU']

In [ ]:
#填充缺失值为0
data_merge_dept_edit=data_merge_dept_edit.fillna(np.nan)
data_merge_dept_edit.isnull().sum()

In [ ]:
# data_merge_dept_edit[['操作费','退货金额','退货数量','退货货品成本']]=data_merge_dept_edit\
# [['操作费','退货金额','退货数量','退货货品成本']].fillna(0)

In [ ]:
# data_merge_dept_edit.to_excel("20190131.xlsx",header=True,index=False)
data_merge_dept_edit.to_excel("data_merge_dept.xlsx",header=True,index=False)

In [ ]:
# data_merge_dept.columns=['Date', 'SKU', 'BarCode', 'Name', 'SellCount', 'SellIncome',
#        'SellCost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost',
#        'ReturnTotal', 'ReturnCount', 'ReturnCost', 'Company', 'Dept',
#        'Group']

In [ ]:
# def merge(data_left,data_right,key):
#     left=pd.read_csv(data_left,header = 0,dtype = object,engine='python',encoding='utf-8')
#     right=pd.read_csv(data_right,header = 0,dtype = object,engine='python')
#     right=right.drop_duplicates(key)
#     data_merge=pd.merge(left,right,on=key,how='left')
#     return data_merge

In [ ]:
# del data['BarCode_x']
# data['BarCode']=data['BarCode_y']
# data=data.drop('BarCode_y',axis=1)